# Práctica 1: Aprendizaje Automático
## Parte A: Evaluación de modelos de Aprendizaje Automático
Hecho por:
- Jaime Benedí
- Miguel Sevilla

En esta parte vamos a utilizar la librería [surprise](https://surprise.readthedocs.io/en/stable/) que implementa algoritmos y otros procedimientos para desarrollar y evaluar algoritmos de sistemas de recomendación. Asegúrate de tenerla instalada antes de empezar a hacer las tareas.

Trabajaremos sobre el dataset de [MovieLens de 100K](https://grouplens.org/datasets/movielens/100k/). MovieLens es un dataset muy usado en el desarrollo y evaluación de sistemas de recomendación, y el dataset de 100k contiene 100K interacciones entre usuarios y productos, almacenando qué usuario ha puntuado qué película y con qué rating.

## Objetivos
1. Cargar MovieLens de 100k.

2. Divide el dataset en una partición simple, donde el conjunto de evaluación sea el 75% de las interacciones y el resto forme parte del conjunto de evaluación. Aquí tenéis que pasarle random_state = SEED a la función.

3. Vamos a evaluar distintos algoritmos de recomendación:
   1. Filtrado colaborativo basado en vecinos (KNNBasic en esta [librería](https://surprise.readthedocs.io/en/stable/knn_inspired.html)) tanto basado en usuarios como enproductos. Utilizaremos la métrica de similitud de Pearson.
   2. Filtrado colaborativo basado en modelos usando [factorización de matrices](https://surprise.readthedocs.io/en/stable/matrix_factorization.html), usando los algoritmos de SVD y NMF.

4. Cada uno de estos algoritmos se entrenarán con el conjunto de
entrenamiento. Aquí tenéis que pasarle random_state = SEED a cada uno de los modelos.

5. Después se obtendrán las predicciones que todos los algoritmos obtienen para el conjunto de evaluación. Muestra el resultado de 5 predicciones e interpreta los resultados.

6. Crea una tabla con los valores que se obtienen para las métricas de evaluación RMSE, precision@k, recall@k, y NDCG, k es el tamaño de la lista de recomendación y será k = 10. Surprise solo implementa RMSE. Las demás las podéis encontrar en sklearn usando precision_score, recall_score, ndcg_score.
   1. **IMPORTANTE**: solo las películas cuyo rating sea superior a 4 serán consideradas relevantes (incluyendo en la lista de películas recomendadas por el modelo).

7. Explica cada uno de los resultados obtenidos y qué significado tienen. Determina cuál podría ser el mejor método recomendador a utilizar.

## Antes de empezar...
Determina un valor para una variable SEED (el que sea). Esta variable se la vamos a pasar a los modelos para que cada vez que se ejecute el código salgan los mismos resultados. De esta forma os aseguráis de que los resultados que os salgan serán los mismos que me salgan a mí al ejecutar el código. Si no hacéis esto, vuestro análisis puede no tener ningún sentido en mi ejecución y os arriesgáis al no apto.

In [13]:
SEED = 42

In [14]:
from surprise import Dataset, SVD
from surprise.model_selection import cross_validate

In [15]:
# 1 -> Cargar el dataset
data = Dataset.load_builtin("ml-100k")

In [16]:
# 2 -> Dividir el dataset: 75% conjunto de evaluación (Posible errata -> ¿Entrenamiento?)
# pasarle random_state = SEED a la función -> facilita la reproducibilidad de los experimentos

from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.25,random_state = SEED)

In [17]:
# 3.1 Algoritmo de recomendación: KNNBasic
from surprise import Dataset, KNNBasic

# Creamos el set de entrenamiento
trainset = data.build_full_trainset()

# Construimos el algoritmo y lo entrenamos
algoKNN = KNNBasic(random_state = SEED)
algoKNN.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [18]:
usuario_id = str(196)  # Identificador del usuario en forma de string
item_id = str(302)  # Identificador del Item en formato de cadena

# Prediccion para usuario e item específico
pred = algoKNN.predict(usuario_id, item_id, r_ui=4, verbose=True)

user: 196        item: 302        r_ui = 4.00   est = 4.06   {'actual_k': 40, 'was_impossible': False}


In [19]:
# Generar predicciones en el conjunto de prueba
predictions_knn = algo.test(testset)

# Calcular el RMSE para evaluar el error del modelo
accuracy.rmse(predictions_knn)


RMSE: 0.7789


0.7788580752369185

RMSE (Root Mean Squared Error) -> mide la precisión del modelo, cuanto menor sea esta mayor sera la capacidad del modelo para hacer sus predicciones. 0.7789 es un valor aceptable , sin embargo para saber cual es el mas adecuado para este dataset vamos a calcular primero el SVD y el NMF para comparar.

In [20]:
# 3.2 Algoritmo de recomendación: Factorización de matrices -> usando los algoritmos de SVD y NMF
from surprise import accuracy

algoSVD = SVD(random_state = SEED)

# Entrenar el algoritmo con el trainset y predecir los ratios para el test
algoSVD.fit(trainset)
predictionsSVD = algoSVD.test(testset)

accuracy.rmse(predictionsSVD)

RMSE: 0.6757


0.6757273833579454

In [21]:
from surprise import NMF

# Crear una instancia del modelo NMF con la misma semilla para comparación
algoNMF = NMF(random_state=SEED)

# Entrenar el modelo en el conjunto de entrenamiento
algoNMF.fit(trainset)

# Predecir las calificaciones para el conjunto de prueba
predictionsNMF = algoNMF.test(testset)

# Calcular el RMSE para evaluar el error del modelo
accuracy.rmse(predictionsNMF)


RMSE: 0.8233


0.8233132889187696

El RMSE empleando este algoritmo (NMF) y el algoritmo KNN son mayores que el de SVD por lo tanto SVD tiene un mejor desempeño en este caso y el algoritmo más recomendado